In [3]:
%matplotlib widget
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne

In [4]:
# Load the data
base_path = base_path = Path.cwd().parent / "data" / "exp1" / "session_20250705_2207"
df_eeg= pd.read_csv(base_path / 'eeg_data.csv')
df_gui = pd.read_csv(base_path / 'gui_data.csv')

In [ ]:
# Time thingy if thats all needed
df_gui['timestamp'] = df_gui['timestamp'] - df_gui['timestamp'].iloc[0]
df_eeg['timestamp'] = df_eeg['timestamp'] - df_gui['timestamp'].iloc[0]

In [ ]:
# EEG: strech time vector to resolve duplicate timestamps





In [3]:
# Estimate sampling frequency (in Hz)
timestamps = df_eeg['timestamp'].values
dt = np.median(np.diff(timestamps))  # assume constant sampling
fs = 1.0 / dt
print(f"Estimated sampling frequency: {fs:.2f} Hz")

Estimated sampling frequency: 32.00 Hz


## Things to See
- **A** – Average time for the Epochs  
- **B** – The electrode graph 

## Preprocessing List
- **A** – Bandpass filter 1–20 Hz 


## The List to Try

### A. Normalization
- **A1** – Demean per epoch, then average  
- **A2** – Average, then demean  
- **A3** – Just average  

### B. Data Usage
- **B1** – Time values between 290 ms to end  
- **B2** – Time values between 290 ms to end, then decimate by 4  
- **B3** – P300 amplitude: peak in 290–500 ms ±10 ms window  
- **B4** – P300 amplitude: max in 290–500 ms  
- **B5** – P300 amplitude at exactly 300 ms  

### C. Feature Selection
- **C1** – Fisher LDA  
- **C2** - Statistically significant electrodes
- **C3** - Manual Feature selection as "M = mean 8 electrode P300 values" and P300/M for normalization

### M. ML Methods
 ? Should I have output as binary and prob of correct or selection from 1-2-3 ? 
- **M1** – LDA  
- **M2** – SVM  
- **M3** – DAWN + LDA  
- **M4** - Random Forest with redacted columns



#### Later
- Try and discard the epochs with eye blinking with abs (max -min) and threshold


In [ ]:
def extract_and_average_epochs_by_stimulus(df_eeg, df_gui, fs=128, post_time=0.6, n_average=0, normalization="A1",blink_channel_idx=0, blink_threshold=120):
    """
    Extracts EEG epochs aligned to GUI stimulus timestamps,
    applies selected normalization (A1, A2, A3),
    and returns the averaged epoch per stimulus.

    Parameters
    ----------
    df_eeg : pd.DataFrame
        EEG data with 'timestamp' column and one column per channel.
    df_gui : pd.DataFrame
        GUI data with 'timestamp' and 'stimulus' columns.
    fs : int or float
        Sampling frequency in Hz.
    post_time : float
        Time after stimulus in seconds.
    n_average : int
        Number of epochs to average per stimulus. 0 means use all.
    normalization : str
        'A1' = Demean each epoch, then average  
        'A2' = Average all epochs, then demean  
        'A3' = Just average without any demeaning

    Returns
    -------
    averaged_epochs : dict
        Dictionary with keys {1, 2, 3}, each containing array of shape (epoch_len, n_channels)
    """
    eeg_X = df_eeg.drop(columns='timestamp').values
    n_samples, n_channels = eeg_X.shape

    epoch_len = int(post_time * fs)
    epoch_offsets = np.arange(epoch_len)[None, :]

    averaged_epochs = {}

    for stim in [1, 2, 3]:
        gui_t = df_gui[df_gui['stimulus'] == stim]['timestamp'].values 
        gui_eeg_idx = np.searchsorted(eeg_t, gui_t)
        valid_mask = (gui_eeg_idx + epoch_len < n_samples)
        gui_eeg_idx = gui_eeg_idx[valid_mask]

        if len(gui_eeg_idx) == 0:
            averaged_epochs[stim] = np.full((epoch_len, n_channels), np.nan)
            continue

        if n_average > 0:
            gui_eeg_idx = gui_eeg_idx[:n_average]

        epochs = eeg_X[gui_eeg_idx[:, None] + epoch_offsets]  # (n_epochs, epoch_len, n_channels)

        # Fast vectorized blink rejection (based on peak-to-peak amplitude)
        blink_channel = epochs[:, :, blink_channel_idx]
        ptp_amplitude = np.ptp(blink_channel, axis=1)
        keep_mask = ptp_amplitude <= blink_threshold
        epochs = epochs[keep_mask]

        if normalization == "A1":
            epoch_mean = epochs.mean(axis=1, keepdims=True)
            epochs_demeaned = epochs - epoch_mean
            averaged = epochs_demeaned.mean(axis=0)

        elif normalization == "A2":
            averaged = epochs.mean(axis=0)
            averaged = averaged - averaged.mean(axis=0, keepdims=True)

        elif normalization == "A3":
            averaged = epochs.mean(axis=0)

        else:
            raise ValueError("Invalid normalization type. Use 'A1', 'A2', or 'A3'.")

        averaged_epochs[stim] = averaged

    return averaged_epochs


In [ ]:
def extract_features_from_averaged_epochs(averaged_epochs, fs=250, feature_type="B1"):
    """
    Extracts features from averaged EEG epochs using different strategies (B1-B5).

    Parameters
    ----------
    averaged_epochs : dict
        Dictionary with keys {1, 2, 3}, each containing a NumPy array of shape (epoch_len, n_channels),
        representing the averaged EEG response for that stimulus.
    fs : int
        Sampling frequency in Hz.
    feature_type : str
        One of the following options:

        - "B1" - Time-series from 290 ms to end of epoch (shape: T × C)
        - "B2" - Same as B1, but decimated by 4 (shape: T/4 × C)
        - "B3" - P300 amplitude: mean ±10 ms around peak in 290-500 ms window (shape: C,)
        - "B4" - P300 amplitude: max value in 290-500 ms (shape: C,)
        - "B5" - P300 amplitude at exactly 310 ms (shape: C,)

    Returns
    -------
    features : dict
        Dictionary with keys {1, 2, 3}, each containing:
        - For B1 and B2: a time - channel matrix (2D array)
        - For B3-B5: a channel-wise summary vector (1D array)
    """
    features = {}
    start_ms = 290
    p300_window = (290, 500)
    window_size_ms = 10
    t_300_ms = 310

    for stim, epoch in averaged_epochs.items():
        if epoch is None or np.all(np.isnan(epoch)):
            features[stim] = None
            continue

        n_time, n_channels = epoch.shape
        time_vector = np.arange(n_time) * (1000 / fs)  # time in ms

        if feature_type == "B1":
            # Time-series from 290 ms onward (shape: T × C)
            mask = time_vector >= start_ms
            feat = epoch[mask]  # shape: (T_b1, C)

        elif feature_type == "B2":
            # Time-series from 290 ms onward, decimated by 4 (shape: T/4 × C)
            mask = time_vector >= start_ms
            feat = epoch[mask][::4]  # shape: (T_b2, C)

        elif feature_type == "B3":
            # Peak in 290–500 → average ±10 ms window around it
            mask = (time_vector >= p300_window[0]) & (time_vector <= p300_window[1])
            windowed = epoch[mask]
            time_in_window = time_vector[mask]
            peak_idx = np.argmax(windowed, axis=0)
            feat = []
            for ch in range(n_channels):
                center_time = time_in_window[peak_idx[ch]]
                win_mask = (time_vector >= center_time - window_size_ms) & (time_vector <= center_time + window_size_ms)
                feat.append(epoch[win_mask, ch].mean())
            feat = np.array(feat)  # shape: (C,)

        elif feature_type == "B4":
            # Max amplitude between 290–500 ms (shape: C,)
            mask = (time_vector >= p300_window[0]) & (time_vector <= p300_window[1])
            feat = epoch[mask].max(axis=0)

        elif feature_type == "B5":
            # Amplitude at exactly 300 ms (shape: C,)
            idx_300 = np.argmin(np.abs(time_vector - t_300_ms))
            feat = epoch[idx_300]

        else:
            raise ValueError("Invalid feature_type. Use 'B1', 'B2', 'B3', 'B4', or 'B5'.")

        features[stim] = feat

    return features


In [ ]:
def process_all_trials(df_eeg, df_gui, fs=128, post_time=0.6, n_average=0, normalization="A1",feature_type="B3", blink_channel_idx=0, blink_threshold=120):
    """
    Iteratively processes all trials in df_gui using extract_and_average_epochs_by_stimulus.
    
    Parameters
    ----------
    df_eeg : pd.DataFrame
        EEG data.
    df_gui : pd.DataFrame
        GUI data with 'trial', 'timestamp', and 'stimulus'.
    
    Returns
    -------
    all_averaged_epochs : list of dicts
        List of averaged epoch dicts from each trial.
    """
    all_averaged_epochs = []
    
    # Group df_gui by each trial
    for trial_id, trial_gui in df_gui.groupby('trial'):

        averaged_epochs = extract_and_average_epochs_by_stimulus(df_eeg=df_eeg,df_gui=trial_gui,fs=fs,post_time=post_time,n_average=n_average,
            normalization=normalization,blink_channel_idx=blink_channel_idx,blink_threshold=blink_threshold)
        
        target = trial_gui['target'].iloc[0] # target for the trial
        features_avg_epoch = extract_features_from_averaged_epochs(averaged_epochs, fs=fs, feature_type=feature_type)

        all_averaged_epochs.append(averaged_epochs)
    
    return all_averaged_epochs


In [ ]:
def build_binary_classification_from_trial(features_dict, target_class):
    """
    Converts multi-class trial into binary classification sample set
    Parameters
    ----------
    features_dict : dict
        Dictionary {class_index: feature_vector (shape C,)} for current trial.
    target_class : int
        Index of the true target class (e.g., 0, 1, or 2)
    Returns
    -------
    X : np.ndarray, shape (3, C)
        Feature matrix for all 3 stimuli.
    y : np.ndarray, shape (3,)
        Binary labels: 1 if class is the target, 0 otherwise.
    """
    keys = sorted(features_dict.keys())  # ensure consistent order
    X = np.vstack([features_dict[k] for k in keys])
    y = np.array([1 if k == target_class else 0 for k in keys])
    return X, y


I need feature selection
    - RF 

I can also try Common Spatial Patterns

In [ ]:
# Feature Selection
from sklearn.feature_selection import SelectKBest, f_classif
# X: shape (n_samples, n_features)
# y: shape (n_samples,), values as 1, 0

selector = SelectKBest(score_func=f_classif, k=10)
X_selected = selector.fit_transform(X, y)
selected_indices = selector.get_support(indices=True)

In [ ]:
# Feature Extraction      
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# X: shape (n_samples, n_features)
# y: shape (n_samples,), values as 1, 0

# Fischer LDA
lda = LinearDiscriminantAnalysis()
X_lda = lda.fit_transform(X, y)   # Reduced features
y_pred = lda.predict(X)           # Class predictions (if needed)



I am doing the feature X and y preparer for B3-B5

In [5]:
import time

start = time.time()
epochs = extract_epochs_by_stimulus(df_eeg, df_gui, fs=250, post_time=0.8)

eeg_epochs_1 = epochs[1]
eeg_epochs_2 = epochs[2]
eeg_epochs_3 = epochs[3]

end = time.time()
print(f"Execution time: {(end - start)*1000:.2f} ms")

Execution time: 11.97 ms


In [ ]:
# Demean the epochs globally
eeg_epochs_1_demeaned = global_demean_epochs(eeg_epochs_1)
eeg_epochs_2_demeaned = global_demean_epochs(eeg_epochs_2)
eeg_epochs_3_demeaned = global_demean_epochs(eeg_epochs_3)
